Get 

In [17]:
import requests
import pandas as pd

API_KEY = '4e1f86110caa98dbcd444fdf6229e002'
FORM_ID = '250216539146152'

def get_field_map():
    url = f'https://api.jotform.com/form/{FORM_ID}/questions?apiKey={API_KEY}'
    response = requests.get(url)
    questions = response.json()['content']
    return {str(qid): qdata['text'] for qid, qdata in questions.items()}

def fetch_jotform_json():
    all_submissions = []
    offset = 0
    while True:
        url = f'https://api.jotform.com/form/{FORM_ID}/submissions?apiKey={API_KEY}&limit=1000&offset={offset}'
        data = requests.get(url).json()
        batch = data['content']
        if not batch:
            break
        all_submissions.extend(batch)
        offset += 1000
    return all_submissions

def clean_jotform_data(submissions, field_map):
    flat_list = []
    for submission in submissions:
        flat = {}
        for k, v in submission['answers'].items():
            label = field_map.get(k, k)
            ans = v.get('answer', None)
            if isinstance(ans, dict):
                for subk, subv in ans.items():
                    flat[f"{label}_{subk}"] = subv
            else:
                flat[label] = ans
        flat_list.append(flat)
    df = pd.DataFrame(flat_list)
    return df

Fech and display the data

In [ ]:
field_map = get_field_map()
submissions = fetch_jotform_json()
df = clean_jotform_data(submissions, field_map)
df.head()  # Show the first few rows

Edit dta

In [ ]:
# Show all columns
df.columns

# Filter by a specific field
df[df['Your Question Text'] == 'Some Value']

# Edit a value
df.at[0, 'Your Question Text'] = 'New Value'

# Save your edits
df.to_csv('edited_jotform_data.csv', index=False)

Rename columns

In [29]:
# Assuming you have already loaded your DataFrame as df

# List of your desired short keys in order
short_keys = [
    "implantacion_registro",
    "medico_implante_otro",
    "institucion_implante_otro",
    "ciudad_implante_otro",
    "implante_dia",
    "implante_mes",
    "implante_ano",
    "fecha_implante",
    "page_break",
    "info_paciente",
    "paciente_nombre",
    "paciente_cedula",
    "nacimiento_dia",
    "nacimiento_mes",
    "nacimiento_ano",
    "fecha_nacimiento",
    "paciente_telefono",
    "info_implante",
    "anexo_garantia",
    "serial_marcapasos",
    "tipo_dispositivo",
    "indicacion",
    "confirmacion",
    "submit",
    "tecnico_asistente",
    "tecnico_asistente_otro",
    "medico_implante",
    "institucion_implante",
    "ciudad_implante",
    "estado_implante",
    "certifico_veracidad",
    "certifico_veracidad_otro"
]

# Assign the new column names
df.columns = short_keys

# Remove columns by name
df = df.drop(['submit', 'page_break', 'confirmacion'], axis=1)


Display column names

In [ ]:
for i, col in enumerate(df.columns):
    print(f"{i}: {col}")

Clean medico_implante_otro

In [ ]:
import unicodedata
import re
from rapidfuzz import process, fuzz

# 1. Improved normalization function
def normalize_name(name):
    if not isinstance(name, str):
        return ''
    # Remove accents
    name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')
    # Lowercase
    name = name.lower()
    # Remove common prefixes (dr, dra, etc.)
    name = re.sub(r'\bdr\.?\b|\bdra\.?\b', '', name)
    # Remove leading/trailing punctuation and spaces
    name = re.sub(r'^[^a-zA-Z0-9]+', '', name)  # Remove leading non-alphanum
    name = re.sub(r'[^a-zA-Z0-9]+$', '', name)  # Remove trailing non-alphanum
    # Remove extra spaces between words
    name = re.sub(r'\s+', ' ', name)
    # Capitalize first letter of each word
    name = name.title()
    # Final strip to remove any leading/trailing spaces
    name = name.strip()
    return name

df['medico_implante_otro_normalized'] = df['medico_implante_otro'].apply(normalize_name)

# 2. Get unique, non-empty normalized names
unique_names = df['medico_implante_otro_normalized'].dropna().unique()
unique_names = [name for name in unique_names if name]

print("All unique normalized names:")
for i, name in enumerate(unique_names, 1):
    print(f"{i}: {name}")

# 3. Fuzzy match similar names (threshold 90%)
print("\nGroups of similar names (fuzzy match >= 90%):")
seen = set()
for name in unique_names:
    if name in seen:
        continue
    matches = process.extract(name, unique_names, scorer=fuzz.ratio, limit=None)
    similar = [match for match, score, _ in matches if score >= 90]
    if len(similar) > 1:
        print(f"Group: {similar}")
        seen.update(similar)

In [ ]:
# Make sure both columns are strings (to avoid NaN issues)
df['medico_implante'] = df['medico_implante'].fillna('').astype(str).str.strip()
df['medico_implante_otro_normalized'] = df['medico_implante_otro_normalized'].fillna('').astype(str).str.strip()

# Create the merged column
df['medico_implante_final'] = df['medico_implante']
df.loc[df['medico_implante_final'] == '', 'medico_implante_final'] = df['medico_implante_otro_normalized']

df = df.drop(['medico_implante', 'medico_implante_otro', 'medico_implante_otro_normalized'], axis=1)

In [41]:
# Convert to datetime if not already
df['fecha_implante'] = pd.to_datetime(df['fecha_implante'], errors='coerce')

# Format as YYYY-MM-DD string (no time)
df['fecha_implante'] = df['fecha_implante'].dt.strftime('%Y-%m-%d')

# Now save to CSV
df.to_csv('cleaned_jotform_data.csv', index=False)

Remove accents

In [ ]:
import unicodedata

def remove_accents(val):
    if isinstance(val, str):
        return unicodedata.normalize('NFKD', val).encode('ASCII', 'ignore').decode('utf-8')
    return val

# Remove accents from all fields in the DataFrame
df = df.applymap(remove_accents)

In [ ]:
df.head(10)

Display installations by date

In [61]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import calendar
# import numpy as np

# # Ensure fecha_implante is datetime
# df['fecha_implante'] = pd.to_datetime(df['fecha_implante'], errors='coerce')

# # Group by week (Monday as the start of the week)
# df['week_start'] = df['fecha_implante'].dt.to_period('W').apply(lambda r: r.start_time)
# df['week_num'] = df['week_start'].dt.isocalendar().week
# df['year'] = df['week_start'].dt.year
# df['month'] = df['week_start'].dt.month

# # Get counts by week
# counts_by_week = df.groupby(['year', 'week_num', 'month'])['fecha_implante'].count().reset_index()
# counts_by_week = counts_by_week.sort_values(['year', 'week_num'])

# # Prepare x labels: "Wk XX"
# x_labels = [f"Wk {wk}" for wk in counts_by_week['week_num']]

# plt.figure(figsize=(16,7))
# bars = sns.barplot(x=x_labels, y=counts_by_week['fecha_implante'], palette='Blues')

# # Annotate each bar with the count value
# for i, count in enumerate(counts_by_week['fecha_implante']):
#     bars.text(i, count + 0.1, str(count), ha='center', va='bottom', fontsize=9, fontweight='bold')

# plt.xlabel('Semana del Año')
# plt.ylabel('Cantidad de Implantes')
# plt.title('Cantidad de Implantes por Semana')

# # Add month labels only once per month, centered under the corresponding weeks
# months = counts_by_week['month'].values
# month_names = [calendar.month_name[m] for m in months]
# unique_months = []
# month_positions = []
# i = 0
# while i < len(months):
#     m = months[i]
#     name = calendar.month_name[m]
#     # Find the range of this month
#     start = i
#     while i + 1 < len(months) and months[i + 1] == m:
#         i += 1
#     end = i
#     center = (start + end) / 2
#     unique_months.append(name)
#     month_positions.append(center)
#     i += 1

# # Add the month names below the x-axis
# ymin = -max(counts_by_week['fecha_implante']) * 0.08
# for pos, name in zip(month_positions, unique_months):
#     plt.text(pos, ymin, name, ha='center', va='top', fontsize=12, fontweight='bold', color='navy')

# plt.xticks(rotation=0)
# plt.tight_layout()
# plt.show()

In [69]:
#Remove columns

columns_to_drop = [
    'implante_dia', 'implante_mes', 'implante_ano', 'info_paciente',
    'implantacion_registro', 'nacimiento_dia', 'nacimiento_mes', 'nacimiento_ano',
    'info_implante', 'mes_implante', 'week_start', 'week_num', 'year', "month"
]

df = df.drop(columns=columns_to_drop, errors='ignore')

# Display the remaining columns to confirm
print(df.columns.tolist())

['institucion_implante_otro', 'ciudad_implante_otro', 'fecha_implante', 'paciente_nombre', 'paciente_cedula', 'fecha_nacimiento', 'paciente_telefono', 'anexo_garantia', 'serial_marcapasos', 'tipo_dispositivo', 'indicacion', 'tecnico_asistente', 'tecnico_asistente_otro', 'institucion_implante', 'ciudad_implante', 'estado_implante', 'certifico_veracidad', 'certifico_veracidad_otro', 'medico_implante_final']


In [71]:
#Reorder columns

desired_order = [
    'institucion_implante',
    'institucion_implante_otro',
    'ciudad_implante',
    'ciudad_implante_otro',
    'estado_implante',
    'fecha_implante',
    'paciente_nombre',
    'paciente_cedula',
    'fecha_nacimiento',
    'paciente_telefono',
    'tipo_dispositivo',
    'serial_marcapasos',
    'indicacion',
    'anexo_garantia',
    'medico_implante_final',
    'tecnico_asistente',
    'tecnico_asistente_otro',
    'certifico_veracidad',
    'certifico_veracidad_otro'
]

# Reorder the DataFrame columns
df = df[desired_order]

# Display the new column order to confirm
print(df.columns.tolist())

['institucion_implante', 'institucion_implante_otro', 'ciudad_implante', 'ciudad_implante_otro', 'estado_implante', 'fecha_implante', 'paciente_nombre', 'paciente_cedula', 'fecha_nacimiento', 'paciente_telefono', 'tipo_dispositivo', 'serial_marcapasos', 'indicacion', 'anexo_garantia', 'medico_implante_final', 'tecnico_asistente', 'tecnico_asistente_otro', 'certifico_veracidad', 'certifico_veracidad_otro']


In [73]:
# Save the DataFrame to a CSV file
df.to_csv('cleaned_jotform_data.csv', index=False)